## EDA of Crime in Oakland CA July 2022-October 2022

### Preparing the data:
 
 hi 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import chi2_contingency


In [2]:
# read the data 
df = pd.read_csv('~/Documents/DataSciProjects/Oakland_Crime/CrimeWatch_Maps_Past_90-Days.csv')
df.head(10)

,CRIMETYPE,DATETIME,CASENUMBER,DESCRIPTION,POLICEBEAT,ADDRESS,CITY,STATE,Location
0,VANDALISM,07/09/2022 02:54:00 AM,22-031310,TAMPER WITH VEHICLE,26X,1000 47TH AV,Oakland,CA,POINT (-122.213999 37.768346)
1,PETTY THEFT,07/11/2022 01:31:00 AM,22-031709,THEFT,77X,2200 32ND AV,Oakland,CA,NaN
2,HOMICIDE,07/12/2022 11:30:00 AM,22-031818,SC UNEXPLAINED DEATH,33X,9300 F ST,Oakland,CA,POINT (-122.179946 37.744408)
3,ROBBERY,07/17/2022 05:03:00 PM,22-032624,ROBBERY-FIREARM,15X,1800 PARK BLVD,Oakland,CA,POINT (-122.25096 37.799143)
4,PETTY THEFT,07/19/2022 02:30:00 PM,22-033016,THEFT,09X,5100 BROADWAY,Oakland,CA,POINT (-122.251678 37.835394)
5,STOLEN VEHICLE,07/20/2022 10:45:00 AM,22-033131,VEHICLE THEFT - AUTO,17Y,1300 BELLA VISTA AV,Oakland,CA,NaN
6,MISDEMEANOR ASSAULT,07/18/2022 10:55:00 AM,22-033186,BATTERY,77X,9800 LOUVAINE AV,Oakland,CA,NaN
7,ROBBERY,07/27/2022 07:58:00 PM,22-034245,"ROBBERY - STRONG ARM (HANDS, FISTS, FEET, ETC.)",03X,500 MADISON ST,Oakland,CA,NaN
8,PETTY THEFT,07/22/2022 03:00:00 PM,22-034417,THEFT,19X,200 8TH AV,Oakland,CA,NaN
9,PETTY THEFT,07/28/2022 05:00:00 PM,22-034490,PETTY THEFT:$50-,06X,3200 MARKET ST,Oakland,CA,POINT (-122.275989 37.822389)


In [3]:
#seeing original counts in columns
df.count()

CRIMETYPE       9874
DATETIME       13941
CASENUMBER     13941
DESCRIPTION    13941
POLICEBEAT     13591
ADDRESS        13939
CITY           13941
STATE          13941
Location       12598
dtype: int64

### Cleaning up dataset

In [4]:
#drop rows where at least one missing element 
df = df.dropna()
df.count()

CRIMETYPE      9190
DATETIME       9190
CASENUMBER     9190
DESCRIPTION    9190
POLICEBEAT     9190
ADDRESS        9190
CITY           9190
STATE          9190
Location       9190
dtype: int64

In [5]:
#drop crime duplicates 
df['CRIMETYPE'].drop_duplicates()

0                                 VANDALISM
2                                  HOMICIDE
3                                   ROBBERY
4                               PETTY THEFT
16                              GRAND THEFT
17                        DOMESTIC VIOLENCE
18                            FORCIBLE RAPE
19                            TOWED VEHICLE
23                                  THREATS
24                              BURG - AUTO
38                      MISDEMEANOR ASSAULT
77                     RECOVERED O/S STOLEN
81                        BURG - COMMERCIAL
83                           STOLEN VEHICLE
88                                NARCOTICS
97                                  WEAPONS
98                           FELONY ASSAULT
147                                   OTHER
183            STOLEN AND RECOVERED VEHICLE
227                      DISORDERLY CONDUCT
233                      BURG - RESIDENTIAL
235                                   ARSON
260                            E

In [6]:
# make categories of crimes for simplifying analysis
dict_offenses = {
'ROBBERY': 'Theft Related',
'PETTY THEFT': 'Theft Related',
'GRAND THEFT' : 'Theft Related',
'BURG - AUTO' : 'Theft Related',
'RECOVERED O/S STOLEN': 'Theft Related',
'BURG - COMMERCIAL': 'Theft Related',
'STOLEN VEHICLE' : 'Theft Related',
'STOLEN AND RECOVERED VEHICLE' : 'Theft Related',
'BURG - RESIDENTIAL' : 'Theft Related',
'EMBEZZLEMENT' : 'Theft Related',
'BURG - OTHER' : 'Theft Related',
'RECOVERED VEHICLE - OAKLAND STOLEN' : 'Theft Related',
'FRAUD' :'Theft Related',
'FORGERY & COUNTERFEITING':'Theft Related',
'VANDALISM' :'Property Damage',
'ARSON': 'Property Damage',
'HOMICIDE':'Violence & Related',
'DOMESTIC VIOLENCE': 'Violence & Related',
'FORCIBLE RAPE': 'Violence & Related',
'MISDEMEANOR ASSAULT':'Violence & Related',
'WEAPONS':'Violence & Related',
'FELONY ASSAULT':'Violence & Related',
'KIDNAPPING': 'Violence & Related',
'CHILD ABUSE':'Violence & Related',
'OTHER SEX OFFENSES': 'Sexual Offenses & Related',
'PROSTITUTION': 'Sexual Offenses & Related',
'THREATS': 'Dangerous practices',
'DISORDERLY CONDUCT':'Dangerous practices',
'BRANDISHING':'Dangerous practices',
'NARCOTICS':'Substances & Related',
'DUI': 'Substances & Related',
'DRUNKENNESS':'Substances & Related',
'GAMBLING':'Substances & Related',
'TOWED VEHICLE':'Other Offenses',
'OTHER':'Other Offenses',
'CURFEW & LOITERING':'Other Offenses',
'MISCELLANEOUS TRAFFIC CRIME':'Other Offenses',
'MISSING':'Other Offenses',
'RUNAWAY':'Other Offenses',
'ENVIRONMENTAL CRIME':'Other Offenses',
'MISDEMEANOR WARRANT' : 'Other Offenses',
'INCIDENT TYPE': 'Other Offenses',
'POSSESSION - STOLEN PROPERTY' : 'Theft Related'}


In [7]:
#add new categories column to df 

df['OFFENSES_CAT']=df['CRIMETYPE'].replace(dict_offenses)

df.head()

,CRIMETYPE,DATETIME,CASENUMBER,DESCRIPTION,POLICEBEAT,ADDRESS,CITY,STATE,Location,OFFENSES_CAT
0,VANDALISM,07/09/2022 02:54:00 AM,22-031310,TAMPER WITH VEHICLE,26X,1000 47TH AV,Oakland,CA,POINT (-122.213999 37.768346),Property Damage
2,HOMICIDE,07/12/2022 11:30:00 AM,22-031818,SC UNEXPLAINED DEATH,33X,9300 F ST,Oakland,CA,POINT (-122.179946 37.744408),Violence & Related
3,ROBBERY,07/17/2022 05:03:00 PM,22-032624,ROBBERY-FIREARM,15X,1800 PARK BLVD,Oakland,CA,POINT (-122.25096 37.799143),Theft Related
4,PETTY THEFT,07/19/2022 02:30:00 PM,22-033016,THEFT,09X,5100 BROADWAY,Oakland,CA,POINT (-122.251678 37.835394),Theft Related
9,PETTY THEFT,07/28/2022 05:00:00 PM,22-034490,PETTY THEFT:$50-,06X,3200 MARKET ST,Oakland,CA,POINT (-122.275989 37.822389),Theft Related


In [8]:
# Make time and date varibles 
df['DATETIME'].head(10)

0     07/09/2022 02:54:00 AM
2     07/12/2022 11:30:00 AM
3     07/17/2022 05:03:00 PM
4     07/19/2022 02:30:00 PM
9     07/28/2022 05:00:00 PM
12    08/05/2022 03:15:00 AM
16    08/19/2022 10:30:00 AM
17    08/19/2022 09:41:00 PM
18    08/21/2022 06:55:00 PM
19    08/23/2022 10:32:00 AM
Name: DATETIME, dtype: object

In [9]:
#Convert to datetime and 24 hour format 
df['DATETIME_parsed'] = pd.to_datetime(df['DATETIME'], format = "%m/%d/%Y %I:%M:%S %p", errors = 'coerce' )

In [10]:
#check outcome of conversion to original DATETIME
df['DATETIME_parsed'].head(20)

0    2022-07-09 02:54:00
2    2022-07-12 11:30:00
3    2022-07-17 17:03:00
4    2022-07-19 14:30:00
9    2022-07-28 17:00:00
12   2022-08-05 03:15:00
16   2022-08-19 10:30:00
17   2022-08-19 21:41:00
18   2022-08-21 18:55:00
19   2022-08-23 10:32:00
23   2022-08-25 12:08:00
24   2022-08-26 12:17:00
30   2022-08-29 22:00:00
38   2022-07-10 15:10:00
44   2022-07-15 10:20:00
52   2022-08-03 14:32:00
54   2022-08-01 12:30:00
56   2022-07-31 14:40:00
75   2022-07-27 07:00:00
76   2022-07-13 04:30:00
Name: DATETIME_parsed, dtype: datetime64[ns]

In [11]:
# Make day of the week variable 
df["DAY_OF_WEEK"] = df["DATETIME_parsed"].dt.day_name()
df["DAY_OF_WEEK"].head(20)

0      Saturday
2       Tuesday
3        Sunday
4       Tuesday
9      Thursday
12       Friday
16       Friday
17       Friday
18       Sunday
19      Tuesday
23     Thursday
24       Friday
30       Monday
38       Sunday
44       Friday
52    Wednesday
54       Monday
56       Sunday
75    Wednesday
76    Wednesday
Name: DAY_OF_WEEK, dtype: object

In [12]:
# Make time of of day variable
df['TIME_OF_DAY'] = df["DATETIME_parsed"].dt.hour.fillna(0).astype('int')
df['TIME_OF_DAY'].head(10)

0      2
2     11
3     17
4     14
9     17
12     3
16    10
17    21
18    18
19    10
Name: TIME_OF_DAY, dtype: int64

In [13]:
#make year variable 
df['YEAR'] = df["DATETIME_parsed"].dt.year.fillna(0).astype('int')
df['YEAR'].head(10)

0     2022
2     2022
3     2022
4     2022
9     2022
12    2022
16    2022
17    2022
18    2022
19    2022
Name: YEAR, dtype: int64

In [14]:
# Make date variable
df['DATE'] = df["DATETIME_parsed"].dt.date
df['DATE'].head(10)

0     2022-07-09
2     2022-07-12
3     2022-07-17
4     2022-07-19
9     2022-07-28
12    2022-08-05
16    2022-08-19
17    2022-08-19
18    2022-08-21
19    2022-08-23
Name: DATE, dtype: object

In [15]:
df["MM_YEAR"] = df["DATETIME_parsed"].dt.strftime('%m-%Y')
df["MM_YEAR"] 

0        07-2022
2        07-2022
3        07-2022
4        07-2022
9        07-2022
          ...   
13933    08-2022
13935    07-2022
13936    07-2022
13939    08-2022
13940    08-2022
Name: MM_YEAR, Length: 9190, dtype: object

In [17]:
#make month + year variable 
df['MONTH'] = df["DATETIME_parsed"].dt.month_name()
df['MONTH'].head(10)

0       July
2       July
3       July
4       July
9       July
12    August
16    August
17    August
18    August
19    August
Name: MONTH, dtype: object

In [18]:
#make month + year variable 
df["MONTH_YEAR"] = df["MONTH"] + ' ' + df["YEAR"].astype(str)
df["MONTH_YEAR"].head(10)

0       July 2022
2       July 2022
3       July 2022
4       July 2022
9       July 2022
12    August 2022
16    August 2022
17    August 2022
18    August 2022
19    August 2022
Name: MONTH_YEAR, dtype: object

In [19]:
## need to add lat and long
df.head()

,CRIMETYPE,DATETIME,CASENUMBER,DESCRIPTION,POLICEBEAT,ADDRESS,CITY,STATE,Location,OFFENSES_CAT,DATETIME_parsed,DAY_OF_WEEK,TIME_OF_DAY,YEAR,DATE,MM_YEAR,MONTH,MONTH_YEAR
0,VANDALISM,07/09/2022 02:54:00 AM,22-031310,TAMPER WITH VEHICLE,26X,1000 47TH AV,Oakland,CA,POINT (-122.213999 37.768346),Property Damage,2022-07-09 02:54:00,Saturday,2,2022,2022-07-09,07-2022,July,July 2022
2,HOMICIDE,07/12/2022 11:30:00 AM,22-031818,SC UNEXPLAINED DEATH,33X,9300 F ST,Oakland,CA,POINT (-122.179946 37.744408),Violence & Related,2022-07-12 11:30:00,Tuesday,11,2022,2022-07-12,07-2022,July,July 2022
3,ROBBERY,07/17/2022 05:03:00 PM,22-032624,ROBBERY-FIREARM,15X,1800 PARK BLVD,Oakland,CA,POINT (-122.25096 37.799143),Theft Related,2022-07-17 17:03:00,Sunday,17,2022,2022-07-17,07-2022,July,July 2022
4,PETTY THEFT,07/19/2022 02:30:00 PM,22-033016,THEFT,09X,5100 BROADWAY,Oakland,CA,POINT (-122.251678 37.835394),Theft Related,2022-07-19 14:30:00,Tuesday,14,2022,2022-07-19,07-2022,July,July 2022
9,PETTY THEFT,07/28/2022 05:00:00 PM,22-034490,PETTY THEFT:$50-,06X,3200 MARKET ST,Oakland,CA,POINT (-122.275989 37.822389),Theft Related,2022-07-28 17:00:00,Thursday,17,2022,2022-07-28,07-2022,July,July 2022


In [20]:
df[['TODROP','LONG', 'LAT']] = df['Location'].str.split(' ', expand=True)
df

,CRIMETYPE,DATETIME,CASENUMBER,DESCRIPTION,POLICEBEAT,ADDRESS,CITY,STATE,Location,OFFENSES_CAT,...,DAY_OF_WEEK,TIME_OF_DAY,YEAR,DATE,MM_YEAR,MONTH,MONTH_YEAR,TODROP,LONG,LAT
0,VANDALISM,07/09/2022 02:54:00 AM,22-031310,TAMPER WITH VEHICLE,26X,1000 47TH AV,Oakland,CA,POINT (-122.213999 37.768346),Property Damage,...,Saturday,2,2022,2022-07-09,07-2022,July,July 2022,POINT,(-122.213999,37.768346)
2,HOMICIDE,07/12/2022 11:30:00 AM,22-031818,SC UNEXPLAINED DEATH,33X,9300 F ST,Oakland,CA,POINT (-122.179946 37.744408),Violence & Related,...,Tuesday,11,2022,2022-07-12,07-2022,July,July 2022,POINT,(-122.179946,37.744408)
3,ROBBERY,07/17/2022 05:03:00 PM,22-032624,ROBBERY-FIREARM,15X,1800 PARK BLVD,Oakland,CA,POINT (-122.25096 37.799143),Theft Related,...,Sunday,17,2022,2022-07-17,07-2022,July,July 2022,POINT,(-122.25096,37.799143)
4,PETTY THEFT,07/19/2022 02:30:00 PM,22-033016,THEFT,09X,5100 BROADWAY,Oakland,CA,POINT (-122.251678 37.835394),Theft Related,...,Tuesday,14,2022,2022-07-19,07-2022,July,July 2022,POINT,(-122.251678,37.835394)
9,PETTY THEFT,07/28/2022 05:00:00 PM,22-034490,PETTY THEFT:$50-,06X,3200 MARKET ST,Oakland,CA,POINT (-122.275989 37.822389),Theft Related,...,Thursday,17,2022,2022-07-28,07-2022,July,July 2022,POINT,(-122.275989,37.822389)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13933,TOWED VEHICLE,08/24/2022 05:40:00 AM,22-038632,BUY/SELL/POSSESS VEHICLE WITH ALTERED NUMBER,32X,1700 99TH AV,Oakland,CA,POINT (-122.165742 37.744617),Other Offenses,...,Wednesday,5,2022,2022-08-24,08-2022,August,August 2022,POINT,(-122.165742,37.744617)
13935,VANDALISM,07/09/2022 11:33:00 PM,22-915304,VANDALISM,04X,2000 FRANKLIN ST,Oakland,CA,POINT (-122.26691 37.808813),Property Damage,...,Saturday,23,2022,2022-07-09,07-2022,July,July 2022,POINT,(-122.26691,37.808813)
13936,HOMICIDE,07/18/2022 05:33:00 AM,22-032697,MURDER:FIRST DEGREE,31Z,10400 PIPPIN ST,Oakland,CA,POINT (-122.176667 37.735632),Violence & Related,...,Monday,5,2022,2022-07-18,07-2022,July,July 2022,POINT,(-122.176667,37.735632)
13939,STOLEN VEHICLE,08/06/2022 11:53:00 PM,22-035896,VEHICLE THEFT - AUTO,08X,400 28TH ST,Oakland,CA,POINT (-122.264966 37.81639),Theft Related,...,Saturday,23,2022,2022-08-06,08-2022,August,August 2022,POINT,(-122.264966,37.81639)


In [21]:
# delete to drop column from df
del df['TODROP']
df

,CRIMETYPE,DATETIME,CASENUMBER,DESCRIPTION,POLICEBEAT,ADDRESS,CITY,STATE,Location,OFFENSES_CAT,DATETIME_parsed,DAY_OF_WEEK,TIME_OF_DAY,YEAR,DATE,MM_YEAR,MONTH,MONTH_YEAR,LONG,LAT
0,VANDALISM,07/09/2022 02:54:00 AM,22-031310,TAMPER WITH VEHICLE,26X,1000 47TH AV,Oakland,CA,POINT (-122.213999 37.768346),Property Damage,2022-07-09 02:54:00,Saturday,2,2022,2022-07-09,07-2022,July,July 2022,(-122.213999,37.768346)
2,HOMICIDE,07/12/2022 11:30:00 AM,22-031818,SC UNEXPLAINED DEATH,33X,9300 F ST,Oakland,CA,POINT (-122.179946 37.744408),Violence & Related,2022-07-12 11:30:00,Tuesday,11,2022,2022-07-12,07-2022,July,July 2022,(-122.179946,37.744408)
3,ROBBERY,07/17/2022 05:03:00 PM,22-032624,ROBBERY-FIREARM,15X,1800 PARK BLVD,Oakland,CA,POINT (-122.25096 37.799143),Theft Related,2022-07-17 17:03:00,Sunday,17,2022,2022-07-17,07-2022,July,July 2022,(-122.25096,37.799143)
4,PETTY THEFT,07/19/2022 02:30:00 PM,22-033016,THEFT,09X,5100 BROADWAY,Oakland,CA,POINT (-122.251678 37.835394),Theft Related,2022-07-19 14:30:00,Tuesday,14,2022,2022-07-19,07-2022,July,July 2022,(-122.251678,37.835394)
9,PETTY THEFT,07/28/2022 05:00:00 PM,22-034490,PETTY THEFT:$50-,06X,3200 MARKET ST,Oakland,CA,POINT (-122.275989 37.822389),Theft Related,2022-07-28 17:00:00,Thursday,17,2022,2022-07-28,07-2022,July,July 2022,(-122.275989,37.822389)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13933,TOWED VEHICLE,08/24/2022 05:40:00 AM,22-038632,BUY/SELL/POSSESS VEHICLE WITH ALTERED NUMBER,32X,1700 99TH AV,Oakland,CA,POINT (-122.165742 37.744617),Other Offenses,2022-08-24 05:40:00,Wednesday,5,2022,2022-08-24,08-2022,August,August 2022,(-122.165742,37.744617)
13935,VANDALISM,07/09/2022 11:33:00 PM,22-915304,VANDALISM,04X,2000 FRANKLIN ST,Oakland,CA,POINT (-122.26691 37.808813),Property Damage,2022-07-09 23:33:00,Saturday,23,2022,2022-07-09,07-2022,July,July 2022,(-122.26691,37.808813)
13936,HOMICIDE,07/18/2022 05:33:00 AM,22-032697,MURDER:FIRST DEGREE,31Z,10400 PIPPIN ST,Oakland,CA,POINT (-122.176667 37.735632),Violence & Related,2022-07-18 05:33:00,Monday,5,2022,2022-07-18,07-2022,July,July 2022,(-122.176667,37.735632)
13939,STOLEN VEHICLE,08/06/2022 11:53:00 PM,22-035896,VEHICLE THEFT - AUTO,08X,400 28TH ST,Oakland,CA,POINT (-122.264966 37.81639),Theft Related,2022-08-06 23:53:00,Saturday,23,2022,2022-08-06,08-2022,August,August 2022,(-122.264966,37.81639)


In [22]:
#Clean up new lat and long columns 
df["LAT"] = df['LAT'].apply(lambda x: x.strip(')'))
df['LONG'] = df['LONG'].apply(lambda x: x.strip('('))
df

,CRIMETYPE,DATETIME,CASENUMBER,DESCRIPTION,POLICEBEAT,ADDRESS,CITY,STATE,Location,OFFENSES_CAT,DATETIME_parsed,DAY_OF_WEEK,TIME_OF_DAY,YEAR,DATE,MM_YEAR,MONTH,MONTH_YEAR,LONG,LAT
0,VANDALISM,07/09/2022 02:54:00 AM,22-031310,TAMPER WITH VEHICLE,26X,1000 47TH AV,Oakland,CA,POINT (-122.213999 37.768346),Property Damage,2022-07-09 02:54:00,Saturday,2,2022,2022-07-09,07-2022,July,July 2022,-122.213999,37.768346
2,HOMICIDE,07/12/2022 11:30:00 AM,22-031818,SC UNEXPLAINED DEATH,33X,9300 F ST,Oakland,CA,POINT (-122.179946 37.744408),Violence & Related,2022-07-12 11:30:00,Tuesday,11,2022,2022-07-12,07-2022,July,July 2022,-122.179946,37.744408
3,ROBBERY,07/17/2022 05:03:00 PM,22-032624,ROBBERY-FIREARM,15X,1800 PARK BLVD,Oakland,CA,POINT (-122.25096 37.799143),Theft Related,2022-07-17 17:03:00,Sunday,17,2022,2022-07-17,07-2022,July,July 2022,-122.25096,37.799143
4,PETTY THEFT,07/19/2022 02:30:00 PM,22-033016,THEFT,09X,5100 BROADWAY,Oakland,CA,POINT (-122.251678 37.835394),Theft Related,2022-07-19 14:30:00,Tuesday,14,2022,2022-07-19,07-2022,July,July 2022,-122.251678,37.835394
9,PETTY THEFT,07/28/2022 05:00:00 PM,22-034490,PETTY THEFT:$50-,06X,3200 MARKET ST,Oakland,CA,POINT (-122.275989 37.822389),Theft Related,2022-07-28 17:00:00,Thursday,17,2022,2022-07-28,07-2022,July,July 2022,-122.275989,37.822389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13933,TOWED VEHICLE,08/24/2022 05:40:00 AM,22-038632,BUY/SELL/POSSESS VEHICLE WITH ALTERED NUMBER,32X,1700 99TH AV,Oakland,CA,POINT (-122.165742 37.744617),Other Offenses,2022-08-24 05:40:00,Wednesday,5,2022,2022-08-24,08-2022,August,August 2022,-122.165742,37.744617
13935,VANDALISM,07/09/2022 11:33:00 PM,22-915304,VANDALISM,04X,2000 FRANKLIN ST,Oakland,CA,POINT (-122.26691 37.808813),Property Damage,2022-07-09 23:33:00,Saturday,23,2022,2022-07-09,07-2022,July,July 2022,-122.26691,37.808813
13936,HOMICIDE,07/18/2022 05:33:00 AM,22-032697,MURDER:FIRST DEGREE,31Z,10400 PIPPIN ST,Oakland,CA,POINT (-122.176667 37.735632),Violence & Related,2022-07-18 05:33:00,Monday,5,2022,2022-07-18,07-2022,July,July 2022,-122.176667,37.735632
13939,STOLEN VEHICLE,08/06/2022 11:53:00 PM,22-035896,VEHICLE THEFT - AUTO,08X,400 28TH ST,Oakland,CA,POINT (-122.264966 37.81639),Theft Related,2022-08-06 23:53:00,Saturday,23,2022,2022-08-06,08-2022,August,August 2022,-122.264966,37.81639


In [23]:
#Removed data with years after 2022
df = df.loc[(df['YEAR'] < 2030)]
df.head(10)

,CRIMETYPE,DATETIME,CASENUMBER,DESCRIPTION,POLICEBEAT,ADDRESS,CITY,STATE,Location,OFFENSES_CAT,DATETIME_parsed,DAY_OF_WEEK,TIME_OF_DAY,YEAR,DATE,MM_YEAR,MONTH,MONTH_YEAR,LONG,LAT
0,VANDALISM,07/09/2022 02:54:00 AM,22-031310,TAMPER WITH VEHICLE,26X,1000 47TH AV,Oakland,CA,POINT (-122.213999 37.768346),Property Damage,2022-07-09 02:54:00,Saturday,2,2022,2022-07-09,07-2022,July,July 2022,-122.213999,37.768346
2,HOMICIDE,07/12/2022 11:30:00 AM,22-031818,SC UNEXPLAINED DEATH,33X,9300 F ST,Oakland,CA,POINT (-122.179946 37.744408),Violence & Related,2022-07-12 11:30:00,Tuesday,11,2022,2022-07-12,07-2022,July,July 2022,-122.179946,37.744408
3,ROBBERY,07/17/2022 05:03:00 PM,22-032624,ROBBERY-FIREARM,15X,1800 PARK BLVD,Oakland,CA,POINT (-122.25096 37.799143),Theft Related,2022-07-17 17:03:00,Sunday,17,2022,2022-07-17,07-2022,July,July 2022,-122.25096,37.799143
4,PETTY THEFT,07/19/2022 02:30:00 PM,22-033016,THEFT,09X,5100 BROADWAY,Oakland,CA,POINT (-122.251678 37.835394),Theft Related,2022-07-19 14:30:00,Tuesday,14,2022,2022-07-19,07-2022,July,July 2022,-122.251678,37.835394
9,PETTY THEFT,07/28/2022 05:00:00 PM,22-034490,PETTY THEFT:$50-,06X,3200 MARKET ST,Oakland,CA,POINT (-122.275989 37.822389),Theft Related,2022-07-28 17:00:00,Thursday,17,2022,2022-07-28,07-2022,July,July 2022,-122.275989,37.822389
12,PETTY THEFT,08/05/2022 03:15:00 AM,22-035617,THEFT,13X,5700 BROADWAY,Oakland,CA,POINT (-122.246956 37.843139),Theft Related,2022-08-05 03:15:00,Friday,3,2022,2022-08-05,08-2022,August,August 2022,-122.246956,37.843139
16,GRAND THEFT,08/19/2022 10:30:00 AM,22-037828,THEFT,34X,8400 INTERNATIONAL BLVD,Oakland,CA,POINT (-122.17775 37.754272),Theft Related,2022-08-19 10:30:00,Friday,10,2022,2022-08-19,08-2022,August,August 2022,-122.17775,37.754272
17,DOMESTIC VIOLENCE,08/19/2022 09:41:00 PM,22-037926,THEFT,26Y,800 69TH AV,Oakland,CA,POINT (-122.198254 37.755532),Violence & Related,2022-08-19 21:41:00,Friday,21,2022,2022-08-19,08-2022,August,August 2022,-122.198254,37.755532
18,FORCIBLE RAPE,08/21/2022 06:55:00 PM,22-038239,THEFT,24X,2000 35TH AV,Oakland,CA,POINT (-122.217828 37.783411),Violence & Related,2022-08-21 18:55:00,Sunday,18,2022,2022-08-21,08-2022,August,August 2022,-122.217828,37.783411
19,TOWED VEHICLE,08/23/2022 10:32:00 AM,22-038331,VEHICLE THEFT - AUTO,07X,2500 WOOD ST,Oakland,CA,POINT (-122.29162 37.821045),Other Offenses,2022-08-23 10:32:00,Tuesday,10,2022,2022-08-23,08-2022,August,August 2022,-122.29162,37.821045


In [24]:
# went from 9190 rows to 9185
df.count()

CRIMETYPE          9185
DATETIME           9185
CASENUMBER         9185
DESCRIPTION        9185
POLICEBEAT         9185
ADDRESS            9185
CITY               9185
STATE              9185
Location           9185
OFFENSES_CAT       9185
DATETIME_parsed    9184
DAY_OF_WEEK        9184
TIME_OF_DAY        9185
YEAR               9185
DATE               9184
MM_YEAR            9184
MONTH              9184
MONTH_YEAR         9184
LONG               9185
LAT                9185
dtype: int64

In [25]:
df.to_csv('Oakland_Data_Clean.csv')